In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("data/daily_sales_french_bakery.csv", parse_dates=["ds"])
df = df.groupby("unique_id").filter(lambda x: len(x) >= 28)
df = df.drop(["unit_price"], axis=1)
df.head()

In [ ]:
plot_series(df=df, ids=["BAGUETTE", "CROISSANT"], palette="viridis")

In [ ]:
plot_series(df=df, ids=["BAGUETTE", "CROISSANT"], max_insample_length=56, palette="viridis")

## Baseline Models

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import Naive, HistoricAverage, WindowAverage, SeasonalNaive

In [ ]:
horizon = 7

In [ ]:
models = [
    Naive(),
    HistoricAverage(),
    WindowAverage(window_size=7),
    SeasonalNaive(season_length=7)
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=df)
preds = sf.predict(h=horizon)

In [ ]:
preds.head() 

In [ ]:
plot_series(df=df, forecasts_df=preds, ids=["BAGUETTE", "CROISSANT"], max_insample_length=28, palette="viridis")

In [ ]:
test = df.groupby("unique_id").tail(7)
train = df.drop(test.index).reset_index(drop=True)

In [ ]:
sf.fit(df=train)
preds = sf.predict(h=horizon)
eval_df = pd.merge(test, preds, "inner", ["ds", "unique_id"])

In [ ]:
evaluation = evaluate(eval_df, metrics=[mae])
evaluation.head()

In [ ]:
evaluation = evaluation.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
evaluation.head()

In [ ]:
plot_series(df=df, forecasts_df=preds, ids=["BAGUETTE", "CROISSANT"], max_insample_length=28, palette="viridis")

In [ ]:
methods = evaluation.columns[1:].tolist()
values = evaluation.iloc[0, 1:].tolist()

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, values)

for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05, f"{value:.3f}", ha="center", va="bottom", fontweight="bold")

plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()

plt.show()

## ARIMA

In [ ]:
from statsforecast.models import AutoARIMA

In [ ]:
models = [
    AutoARIMA(seasonal=False, alias="ARIMA"),
    AutoARIMA(season_length=7, alias="SARIMA")
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=train)
arima_preds = sf.predict(h=horizon)
arima_eval_df = pd.merge(arima_preds, eval_df, "inner", ["ds", "unique_id"])

In [ ]:
arima_eval = evaluate(arima_eval_df, metrics=[mae])
arima_eval.head()

In [ ]:
arima_eval = arima_eval.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
arima_eval.head()

In [ ]:
plot_series(df=df, forecasts_df=arima_preds, ids=["BAGUETTE", "CROISSANT"], max_insample_length=28, palette="viridis")

In [ ]:
methods = arima_eval.columns[1:].tolist()
values = arima_eval.iloc[0, 1:].tolist()

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, values)

for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05, f"{value:.3f}", ha="center", va="bottom", fontweight="bold")

plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()

plt.show()

## Cross-validation

In [ ]:
models = [
    SeasonalNaive(season_length=7),
    AutoARIMA(seasonal=False, alias="ARIMA"),
    AutoARIMA(season_length=7, alias="SARIMA")
]

sf = StatsForecast(models=models, freq="D")
cv_df = sf.cross_validation(h=horizon, df=df, n_windows=7, step_size=horizon, refit=True)
cv_df.head()

In [ ]:
cv_eval = evaluate(cv_df.drop(["cutoff"], axis=1), metrics=[mae])
cv_eval.head()

In [ ]:
cv_eval = cv_eval.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
cv_eval.head()

In [ ]:
plot_series(df=df, forecasts_df=cv_df.drop(["cutoff", "y"], axis=1), ids=["BAGUETTE", "CROISSANT"], max_insample_length=140, palette="viridis")

In [ ]:
methods = cv_eval.columns[1:].tolist()
values = cv_eval.iloc[0, 1:].tolist()

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, values)

for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05, f"{value:.3f}", ha="center", va="bottom", fontweight="bold")

plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()

plt.show()

## Exogenous Features

In [ ]:
df = pd.read_csv("data/daily_sales_french_bakery.csv", parse_dates=["ds"])
df = df.groupby("unique_id").filter(lambda x: len(x) >= 28)
df.head()

In [ ]:
baguette_plot_df = df[df["unique_id"] == "BAGUETTE"]
croissant_plot_df = df[df["unique_id"] == "CROISSANT"]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 8))

ax1.plot(baguette_plot_df["ds"], baguette_plot_df["y"])
ax1.set_xlabel("Date")
ax1.set_ylabel("Baguette Sales Volume")

ax2.plot(baguette_plot_df["ds"], baguette_plot_df["unit_price"])
ax2.set_xlabel("Date")
ax2.set_ylabel("Unit Price of Baguette")

ax3.plot(croissant_plot_df["ds"], croissant_plot_df["y"])
ax3.set_xlabel("Date")
ax3.set_ylabel("Croissant Sales Volume")

ax4.plot(croissant_plot_df["ds"], croissant_plot_df["unit_price"])
ax4.set_xlabel("Date")
ax4.set_ylabel("Unit Price of Croissant")

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
test = df.groupby("unique_id").tail(7)
train = df.drop(test.index).reset_index(drop=True)

In [ ]:
futr_exog_df = test.drop(["y"], axis=1)
futr_exog_df.head()

In [ ]:
models = [
    AutoARIMA(season_length=7, alias="SARIMA_price_exog")
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=train)
arima_exog_preds = sf.predict(h=horizon, X_df=futr_exog_df)

models = [
    AutoARIMA(season_length=7, alias="SARIMA")
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=train.drop(["unit_price"], axis=1))
arima_preds = sf.predict(h=horizon)

In [ ]:
test_df = test.merge(arima_exog_preds, on=["unique_id", "ds"], how="inner").merge(arima_preds, on=["unique_id", "ds"], how="inner")
test_df.head()

In [ ]:
arima_exog_eval = evaluate(test_df.drop(["unit_price"], axis=1), metrics=[mae])
arima_exog_eval.head()

In [ ]:
arima_exog_eval = arima_exog_eval.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
arima_exog_eval.head()

In [ ]:
plot_series(df=train, forecasts_df=test_df, ids=["BAGUETTE", "CROISSANT"], max_insample_length=28, models=["SARIMA_price_exog", "SARIMA"], palette="viridis")

In [ ]:
methods = arima_exog_eval.columns[1:].tolist()
values = arima_exog_eval.iloc[0, 1:].tolist()

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, values)

for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05, f"{value:.3f}", ha="center", va="bottom", fontweight="bold")

plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()

plt.show()

In [ ]:
models = [
    AutoARIMA(season_length=7, alias="SARIMA_price_exog"),
]

sf = StatsForecast(models=models, freq="D")

cv_exog_df = sf.cross_validation(h=horizon, df=df, n_windows=7, step_size=horizon, refit=True)
cv_exog_df.head()

In [ ]:
cv_exog_df = cv_exog_df.merge(cv_df.drop(["cutoff", "y"], axis=1), on=["unique_id", "ds"], how="inner")
cv_exog_df.head()

In [ ]:
cv_exog_eval = evaluate(cv_exog_df.drop(["cutoff"], axis=1), metrics=[mae])
cv_exog_eval.head()

In [ ]:
cv_exog_eval = cv_exog_eval.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
cv_exog_eval.head()

In [ ]:
plot_series(df=df, forecasts_df=cv_exog_df.drop(["cutoff", "y"], axis=1), ids=["BAGUETTE", "CROISSANT"], max_insample_length=140, palette="viridis")

In [ ]:
methods = cv_exog_eval.columns[1:].tolist()
values = cv_exog_eval.iloc[0, 1:].tolist()

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, values)

for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05, f"{value:.3f}", ha="center", va="bottom", fontweight="bold")

plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()

plt.show()

In [ ]:
from functools import partial
from utilsforecast.feature_engineering import fourier, time_features, pipeline

In [ ]:
features = [
    partial(fourier, season_length=7, k=2),
    partial(time_features, features=["day", "week", "month"])
]

exog_df, futr_exog_df = pipeline(df=df, features=features, freq="D", h=horizon)

In [ ]:
exog_df.head()

In [ ]:
futr_exog_df.head()

In [ ]:
models = [
    AutoARIMA(season_length=7, alias="SARIMA_time_exog")
]

sf = StatsForecast(models=models, freq="D")
cv_time_exog_df = sf.cross_validation(h=horizon, df=exog_df, n_windows=7, step_size=horizon, refit=True)
cv_time_exog_df.head()

In [ ]:
cv_time_exog_df = cv_time_exog_df.merge(cv_exog_df.drop(["cutoff", "y"], axis=1), on=["unique_id", "ds"], how="inner")
cv_time_exog_df.head()

In [ ]:
cv_time_exog_eval = evaluate(cv_time_exog_df.drop(["cutoff"], axis=1), metrics=[mae])
cv_time_exog_eval.head()

In [ ]:
cv_time_exog_eval = cv_time_exog_eval.drop(["unique_id"], axis=1).groupby("metric").mean().reset_index()
cv_time_exog_eval.head()

In [ ]:
plot_series(df=df, forecasts_df=cv_time_exog_df.drop(["cutoff", "y"], axis=1), ids=["BAGUETTE", "CROISSANT"], max_insample_length=140, palette="viridis")

In [ ]:
methods = cv_time_exog_eval.columns[1:].tolist()
values = cv_time_exog_eval.iloc[0, 1:].tolist()

plt.figure(figsize=(10, 6))
bars = plt.bar(methods, values)

for bar, value in zip(bars, values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05, f"{value:.3f}", ha="center", va="bottom", fontweight="bold")

plt.xlabel("Methods")
plt.ylabel("Mean Absolute Error (MAE)")
plt.tight_layout()

plt.show()

## Prediction Intervals

In [ ]:
df = pd.read_csv("data/daily_sales_french_bakery.csv", parse_dates=["ds"])
df = df.groupby("unique_id").filter(lambda x: len(x) >= 28)
df = df.drop(["unit_price"], axis=1)
df.head()

In [ ]:
test = df.groupby("unique_id").tail(7)
train = df.drop(test.index).reset_index(drop=True)

In [ ]:
models = [
    AutoARIMA(season_length=7)
]

sf = StatsForecast(models=models, freq="D")
sf.fit(df=train)
prob_preds = sf.predict(h=horizon, level=[80])
prob_preds.head()

In [ ]:
test_df = test.merge(prob_preds, on=["unique_id", "ds"], how="inner")
test_df.head()

In [ ]:
plot_series(df=train, forecasts_df=test_df, ids=["BAGUETTE", "CROISSANT"], max_insample_length=28, models=["AutoARIMA"], level=[80], palette="viridis")

In [ ]:
models = [
    AutoARIMA(season_length=7)
]

sf = StatsForecast(models=models, freq="D")

cv_prob_df = sf.cross_validation(h=horizon, df=df, n_windows=7, step_size=horizon, refit=True, level=[80])
cv_prob_df.head()

In [ ]:
plot_series(df=train, forecasts_df=cv_prob_df.drop(["y", "cutoff"], axis=1), ids=["BAGUETTE", "CROISSANT"], max_insample_length=140, models=["AutoARIMA"], level=[80], palette="viridis")

## Evaluation Metrics

In [ ]:
df = pd.read_csv("data/daily_sales_french_bakery.csv", parse_dates=["ds"])
df = df.groupby("unique_id").filter(lambda x: len(x) >= 28)
df = df.drop(["unit_price"], axis=1)
df.head()

In [ ]:
models = [
    AutoARIMA(season_length=7, alias="SARIMA"),
    SeasonalNaive(season_length=7)
]

sf = StatsForecast(models=models, freq="D")
final_cv_df = sf.cross_validation(h=horizon, df=df, n_windows=7, step_size=horizon, refit=True, level=[80])
final_cv_df.head()

In [ ]:
temp_test = df.groupby("unique_id").tail(7 * 7)
temp_train = df.drop(temp_test.index).reset_index(drop=True)

In [ ]:
models = ["SARIMA", "SeasonalNaive"]
metrics = [
    mae,
    mse,
    rmse,
    mape,
    smape,
    partial(mase, seasonality=7),
    scaled_crps
]

final_eval = evaluate(final_cv_df.drop(["ds", "cutoff"], axis=1), metrics=metrics, models=models, train_df=temp_train, level=[80])
final_eval.head()

In [ ]:
final_eval = final_eval.drop(["unique_id"], axis=1).groupby(["metric"]).mean().reset_index()
final_eval.head()

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(18, 12))
axes_flat = axes.flatten()

x_pos = [0, 1]
colors = ["blue", "red"]
models = ["SARIMA", "SeasonalNaive"]

for i, row in final_eval.iterrows():
    ax = axes_flat[i]
    model_values = [row["SARIMA"], row["SeasonalNaive"]]
    bars = ax.bar(x_pos, model_values, color=colors, alpha=0.8, edgecolor="black", linewidth=1)
    for j, (bar, value) in enumerate(zip(bars, model_values)):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + bar.get_height() * 0.01, f"{value:.3f}", ha="center", va="bottom", fontweight="bold", fontsize=10)
    
    ax.set_title(row["metric"].upper(), fontweight="bold", fontsize=12)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(models, ha="center")
    ax.set_ylabel("Value")
    max_value = max(model_values)
    ax.set_ylim(0, max_value * 1.1)

fig.delaxes(axes_flat[7])
axes_flat[8].set_visible(False)

plt.tight_layout()
plt.show()
